In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import pickle
import yaml
import pandas as pd
from src import paths
import reservoirpy as rpy


In [64]:
rpy.verbosity(0)

0

In [65]:

# Paths

trained_esn_path = paths.models_dir("trained-esn-non-feedback.pickle")
params_path = paths.config_dir("params.yaml")

pre_megadrought_fault_detection_dataset_path = paths.data_processed_dir("fault_detection", "pre_megadrought_fault_detection_dataset.csv")
pre_megadrought_fault_detection_metadata_path = paths.data_processed_dir("fault_detection", "pre_megadrought_fault_detection_metadata.csv")

In [66]:
with open(params_path, "r") as file:
  params = yaml.safe_load(file)

save_interval: int = params["save_interval"]
weeks_after_change_offset: int = params["weeks_after_change_offset"]
esn_features_dim: int = params["esn_features_dim"]
fault_detection_forecasted_steps: int = params["fault_detection_forecasted_steps"]


In [67]:
date_offset = pd.DateOffset(
    weeks=weeks_after_change_offset
)

In [68]:
pre_megadrought_fault_detection_df = pd.read_csv(pre_megadrought_fault_detection_dataset_path, index_col=["ID", "IDpix"])
pre_megadrought_fault_detection_df.columns = pd.to_datetime(pre_megadrought_fault_detection_df.columns)

pre_megadrought_fault_detection_metadata_df = pd.read_csv(pre_megadrought_fault_detection_metadata_path, index_col=["ID", "IDpix"])

In [69]:
## Detection of event date
from src.features.slice_time_series import create_training_data
from src.features.denoise_ndvi_signal import (
    downsample_time_series,
    moving_std_filter,
    holt_winters_filter,
    denoise_ndvi_signal,
)
from src.models.forecasting import Forecaster


In [70]:
with open(trained_esn_path, "rb") as file:
  model = pickle.load(file)

In [72]:

signal = pre_megadrought_fault_detection_df.iloc[0]
denoised_signal_series = denoise_ndvi_signal(
        signal, [
            downsample_time_series,
            moving_std_filter,
            holt_winters_filter,
        ]
    )
dates = denoised_signal_series.index

y_dates = dates[esn_features_dim:].to_numpy()
y_dates = y_dates.reshape(-1, 1)

X, y = create_training_data(denoised_signal_series.to_numpy(), num_features=esn_features_dim)
y = y.reshape(-1, 1)


In [73]:
T = X.shape[0]
T


914

In [74]:
s = T - fault_detection_forecasted_steps
s

862

In [75]:
Xtrain_signal = X[:s]
ytrain_signal = y[:s]

Xtrain_signal.shape

(862, 104)

In [76]:
signal_forecaster = Forecaster(model, num_features=esn_features_dim)
signal_forecaster.train(Xtrain_signal, ytrain_signal)

In [77]:
warmup_X_signal = Xtrain_signal[-esn_features_dim:, :]
lower_bound = signal_forecaster.forecast(
    prediction_length=T - s, 
    warmup_X=warmup_X_signal,
)

In [78]:
lower_bound_dates = y_dates[s:]

In [79]:
Xtrain_signal.shape

(862, 104)

In [80]:
lower_bound_dates.shape

(52, 1)

In [81]:
lower_bound.shape

(52, 1)